Using kernel `conda_pytorch_latest_p36`

In [1]:
import sys
sys.path.append('../../../')

In [2]:
from pathlib import Path
import os
import random

In [3]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
import torch

In [4]:
from deep.constants import *

In [5]:
%load_ext autoreload
%autoreload 2

## Data

In [6]:
data = FRAMEWORKS_PATH

In [7]:
afexportable = pd.read_csv(data /'afexportable_of_af_of_projects_of_interest.csv')
all_afs = pd.read_csv(data /'all_afs.csv')

proj_interest = pd.read_csv(data / 'projects_of_interest.csv')
entr_proj_interest = pd.read_csv(data / 'entries_of_projects_of_interest.csv')
exp_proj_interest = pd.read_csv(data / 'exportdata_of_entries_of_projects_of_interest.csv')
wid_proj_interest = pd.read_csv(data / 'widgets_of_afs_of_interest.csv')

/Users/stefano/miniconda3/envs/deep-exp/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [8]:
train = pd.read_csv(FRAMEWORKS_PATH / 'data_v0.3_train.csv')
val = pd.read_csv(FRAMEWORKS_PATH / 'data_v0.3_val.csv')
test = pd.read_csv(FRAMEWORKS_PATH / 'data_v0.3_test.csv')

/Users/stefano/miniconda3/envs/deep-exp/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [9]:
columns_to_keep = ['excerpt'] + DIMENSION_CLASSES

In [10]:
def process_for_sector(df, sector):
    df = df[['excerpt', sector]]
        
    return df

In [11]:
train_df = process_for_sector(train, 'Humanitarian Conditions')
test_df = process_for_sector(test, 'Humanitarian Conditions')

In [12]:
sample = False

if sample:
    train_df.sample(1000).to_pickle('train_df.pickle', protocol=4)
    test_df.sample(1000).to_pickle('test_df.pickle', protocol=4)
else:
    train_df.to_pickle('train_df.pickle', protocol=4)
    test_df.to_pickle('test_df.pickle', protocol=4)

## Sagemaker Prep

In [13]:
import sagemaker
from sagemaker import get_execution_role
import json
import boto3

default_bucket = 'deep-experiments-sagemaker-bucket'
sess = sagemaker.Session(default_bucket=default_bucket)

role = 'AmazonSageMaker-ExecutionRole-20210519T102514'
print(
    role
)  # This is the role that SageMaker would use to leverage AWS resources (S3, CloudWatch) on your behalf

bucket = SAGEMAKER_BUCKET
prefix = "huggingface/first"  # Replace with the prefix under which you want to store the data if needed


AmazonSageMaker-ExecutionRole-20210519T102514


### Bucket upload

In [14]:
bucket_path = 'test1/data'
train_channel = bucket_path + "/train_df.pickle"
validation_channel = bucket_path + "/test_df.pickle"

sess.upload_data(path="train_df.pickle", bucket=SAGEMAKER_BUCKET, key_prefix=bucket_path)
sess.upload_data(path="test_df.pickle", bucket=SAGEMAKER_BUCKET, key_prefix=bucket_path)

s3_train_data = f"s3://{SAGEMAKER_BUCKET}/{train_channel}"
s3_validation_data = f"s3://{SAGEMAKER_BUCKET}/{validation_channel}"

s3_output_location = f"s3://{SAGEMAKER_BUCKET}/{bucket_path}/output"

In [15]:
instances = [
    'ml.p2.xlarge',
    'ml.p3.2xlarge'
]

In [16]:
metric_definitions=[
    {'Name': 'loss', 'Regex': "'loss': ([0-9]+(.|e\-)[0-9]+),?"},
    {'Name': 'learning_rate', 'Regex': "'learning_rate': ([0-9]+(.|e\-)[0-9]+),?"},
    {'Name': 'eval_loss', 'Regex': "'eval_loss': ([0-9]+(.|e\-)[0-9]+),?"},
    {'Name': 'eval_accuracy', 'Regex': "'eval_accuracy': ([0-9]+(.|e\-)[0-9]+),?"},
    {'Name': 'eval_f1', 'Regex': "'eval_f1': ([0-9]+(.|e\-)[0-9]+),?"},
    {'Name': 'eval_precision', 'Regex': "'eval_precision': ([0-9]+(.|e\-)[0-9]+),?"},
    {'Name': 'eval_recall', 'Regex': "'eval_recall': ([0-9]+(.|e\-)[0-9]+),?"},
    {'Name': 'eval_stupid_metric', 'Regex': "'eval_stupid_metric': ([0-9]+(.|e\-)[0-9]+),?"},
    {'Name': 'eval_runtime', 'Regex': "'eval_runtime': ([0-9]+(.|e\-)[0-9]+),?"},
    {'Name': 'eval_samples_per_second', 'Regex': "'eval_samples_per_second': ([0-9]+(.|e\-)[0-9]+),?"},
    {'Name': 'epoch', 'Regex': "'epoch': ([0-9]+(.|e\-)[0-9]+),?"}]

In [17]:
# # set True if you need spot instance
# use_spot = True
# train_max_run_secs =   2*24 * 60 * 60
# spot_wait_sec =  5 * 60
# max_wait_time_secs = train_max_run_secs +  spot_wait_sec

# if not use_spot:
#     max_wait_time_secs = None
    
# # During local mode, no spot.., use smaller dataset
# if instance_type == 'local':
#     use_spot = False
#     max_wait_time_secs = 0
#     wait = True
#     # Use smaller dataset to run locally
#     inputs = inputs_sample


In [18]:
from sagemaker.pytorch import PyTorch

hyperparameters={'epochs': 1,
                 'train_batch_size': 32,
                 'model_name': 'distilbert-base-uncased'
                 }

estimator = PyTorch(
    entry_point='train.py',
    source_dir=str(SCRIPTS_TRAINING_PATH / 'stefano/pytorch_estimator_base'),
    output_path=f's3://{SAGEMAKER_BUCKET}/models/',
    instance_type='ml.p3.2xlarge',
    instance_count=1,
    role=role,
    framework_version='1.8',
    py_version='py36',
    hyperparameters = hyperparameters,
    metric_definitions=metric_definitions
#     train_instance_count=2,
#     train_instance_type="ml.c4.xlarge",
)

In [19]:
fit_arguments = {
    'train': f's3://{SAGEMAKER_BUCKET}/{bucket_path}',
    'test': f's3://{SAGEMAKER_BUCKET}/{bucket_path}'
}

In [20]:
estimator.fit(fit_arguments)

2021-05-27 12:23:57 Starting - Starting the training job...
2021-05-27 12:24:21 Starting - Launching requested ML instancesProfilerReport-1622118235: InProgress
......
2021-05-27 12:25:21 Starting - Preparing the instances for training.........
2021-05-27 12:27:01 Downloading - Downloading input data
2021-05-27 12:27:01 Training - Downloading the training image....................bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-05-27 12:30:31,481 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2021-05-27 12:30:31,506 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-05-27 12:30:37,791 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-05-27 12:30:38,147 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/opt/conda/bin/python3.6 -m pip install -r requirements.txt

2021-05-

2021-05-27 12:30:56,144 - filelock - INFO - Lock 140698670207608 acquired on /root/.cache/huggingface/transformers/8c8624b8ac8aa99c60c912161f8332de003484428c47906d7ff7eb7f73eecdbb.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79.lock
2021-05-27 12:30:56,169 - filelock - INFO - Lock 140698670207608 released on /root/.cache/huggingface/transformers/8c8624b8ac8aa99c60c912161f8332de003484428c47906d7ff7eb7f73eecdbb.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79.lock
2021-05-27 12:30:56,229 - filelock - INFO - Lock 140697715412440 acquired on /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.d423bdf2f58dc8b77d5f5d18028d7ae4a72dcfd8f468e81fe979ada957a8c361.lock
2021-05-27 12:30:56,250 - filelock - INFO - Lock 140697715412440 released on /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.d423bdf2f58dc8b77d5f5d18028d7ae4a72dcfd8f468e81fe979ada957a8c361.lock


{'loss': 0.6879, 'learning_rate': 6.000000000000001e-07, 'epoch': 0.0}
{'loss': 0.7229, 'learning_rate': 7.000000000000001e-07, 'epoch': 0.0}
{'loss': 0.7081, 'learning_rate': 8.000000000000001e-07, 'epoch': 0.0}
{'loss': 0.7002, 'learning_rate': 9e-07, 'epoch': 0.0}
{'loss': 0.7079, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.0}
{'loss': 0.6896, 'learning_rate': 1.1e-06, 'epoch': 0.0}
{'loss': 0.7065, 'learning_rate': 1.2000000000000002e-06, 'epoch': 0.0}
{'loss': 0.661, 'learning_rate': 1.3e-06, 'epoch': 0.0}
{'loss': 0.6675, 'learning_rate': 1.4000000000000001e-06, 'epoch': 0.01}
{'loss': 0.6934, 'learning_rate': 1.5e-06, 'epoch': 0.01}
{'loss': 0.7046, 'learning_rate': 1.6000000000000001e-06, 'epoch': 0.01}
{'loss': 0.7221, 'learning_rate': 1.7000000000000002e-06, 'epoch': 0.01}
{'loss': 0.6878, 'learning_rate': 1.8e-06, 'epoch': 0.01}
{'loss': 0.7073, 'learning_rate': 1.9e-06, 'epoch': 0.01}
{'loss': 0.7026, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.01}
{'loss': 

{'loss': 0.5794, 'learning_rate': 1.29e-05, 'epoch': 0.05}
{'loss': 0.7089, 'learning_rate': 1.3000000000000001e-05, 'epoch': 0.05}
{'loss': 0.6738, 'learning_rate': 1.3100000000000002e-05, 'epoch': 0.05}
{'loss': 0.6128, 'learning_rate': 1.32e-05, 'epoch': 0.05}
{'loss': 0.701, 'learning_rate': 1.3300000000000001e-05, 'epoch': 0.05}
{'loss': 0.7373, 'learning_rate': 1.3400000000000002e-05, 'epoch': 0.05}
{'loss': 0.7013, 'learning_rate': 1.3500000000000001e-05, 'epoch': 0.05}
{'loss': 0.5766, 'learning_rate': 1.3600000000000002e-05, 'epoch': 0.05}
{'loss': 0.5476, 'learning_rate': 1.3700000000000001e-05, 'epoch': 0.05}
{'loss': 0.7085, 'learning_rate': 1.3800000000000002e-05, 'epoch': 0.05}
{'loss': 0.4942, 'learning_rate': 1.3900000000000002e-05, 'epoch': 0.05}
{'loss': 0.5248, 'learning_rate': 1.4000000000000001e-05, 'epoch': 0.05}
{'loss': 0.6343, 'learning_rate': 1.4099999999999999e-05, 'epoch': 0.05}
{'loss': 0.6305, 'learning_rate': 1.42e-05, 'epoch': 0.05}
{'loss': 0.6031, 'lea

{'loss': 0.6305, 'learning_rate': 2.4200000000000002e-05, 'epoch': 0.09}
{'loss': 0.6689, 'learning_rate': 2.43e-05, 'epoch': 0.09}
{'loss': 0.4914, 'learning_rate': 2.44e-05, 'epoch': 0.09}
{'loss': 0.5446, 'learning_rate': 2.45e-05, 'epoch': 0.09}
{'loss': 0.7668, 'learning_rate': 2.46e-05, 'epoch': 0.09}
{'loss': 0.5661, 'learning_rate': 2.47e-05, 'epoch': 0.09}
{'loss': 0.7376, 'learning_rate': 2.48e-05, 'epoch': 0.09}
{'loss': 0.4676, 'learning_rate': 2.4900000000000002e-05, 'epoch': 0.09}
{'loss': 0.5077, 'learning_rate': 2.5e-05, 'epoch': 0.09}
{'loss': 0.5982, 'learning_rate': 2.51e-05, 'epoch': 0.09}
{'loss': 0.5249, 'learning_rate': 2.5200000000000003e-05, 'epoch': 0.09}
{'loss': 0.5523, 'learning_rate': 2.5300000000000002e-05, 'epoch': 0.09}
{'loss': 0.5299, 'learning_rate': 2.54e-05, 'epoch': 0.09}
{'loss': 0.6557, 'learning_rate': 2.5500000000000003e-05, 'epoch': 0.09}
{'loss': 0.5706, 'learning_rate': 2.5600000000000002e-05, 'epoch': 0.1}
{'loss': 0.5651, 'learning_rate':

...

{'loss': 0.3796, 'learning_rate': 8.668801463860933e-06, 'epoch': 0.86}
{'loss': 0.592, 'learning_rate': 8.645928636779506e-06, 'epoch': 0.86}
{'loss': 0.4297, 'learning_rate': 8.623055809698079e-06, 'epoch': 0.86}
{'loss': 0.5374, 'learning_rate': 8.600182982616652e-06, 'epoch': 0.86}
{'loss': 0.3459, 'learning_rate': 8.577310155535225e-06, 'epoch': 0.86}
{'loss': 0.5339, 'learning_rate': 8.554437328453797e-06, 'epoch': 0.86}
{'loss': 0.4756, 'learning_rate': 8.53156450137237e-06, 'epoch': 0.86}
{'loss': 0.3932, 'learning_rate': 8.508691674290943e-06, 'epoch': 0.86}
{'loss': 0.5696, 'learning_rate': 8.485818847209517e-06, 'epoch': 0.86}
{'loss': 0.4708, 'learning_rate': 8.462946020128089e-06, 'epoch': 0.86}
{'loss': 0.4385, 'learning_rate': 8.440073193046661e-06, 'epoch': 0.86}
{'loss': 0.5816, 'learning_rate': 8.417200365965233e-06, 'epoch': 0.86}
{'loss': 0.2511, 'learning_rate': 8.394327538883807e-06, 'epoch': 0.86}
{'loss': 0.4559, 'learning_rate': 8.371454711802379e-06, 'epoch': 

{'loss': 0.4424, 'learning_rate': 6.10704483074108e-06, 'epoch': 0.9}
{'loss': 0.388, 'learning_rate': 6.084172003659653e-06, 'epoch': 0.9}
{'loss': 0.4604, 'learning_rate': 6.061299176578225e-06, 'epoch': 0.9}
{'loss': 0.4028, 'learning_rate': 6.038426349496798e-06, 'epoch': 0.9}
{'loss': 0.4508, 'learning_rate': 6.015553522415371e-06, 'epoch': 0.9}
{'loss': 0.5921, 'learning_rate': 5.992680695333944e-06, 'epoch': 0.9}
{'loss': 0.3897, 'learning_rate': 5.969807868252517e-06, 'epoch': 0.9}
{'loss': 0.2716, 'learning_rate': 5.946935041171089e-06, 'epoch': 0.9}
{'loss': 0.4936, 'learning_rate': 5.924062214089662e-06, 'epoch': 0.9}
{'loss': 0.341, 'learning_rate': 5.901189387008235e-06, 'epoch': 0.9}
{'loss': 0.3906, 'learning_rate': 5.878316559926807e-06, 'epoch': 0.9}
{'loss': 0.3494, 'learning_rate': 5.85544373284538e-06, 'epoch': 0.9}
{'loss': 0.3216, 'learning_rate': 5.832570905763952e-06, 'epoch': 0.91}
{'loss': 0.5822, 'learning_rate': 5.809698078682525e-06, 'epoch': 0.91}
{'loss':

{'loss': 0.5105, 'learning_rate': 3.362305580969808e-06, 'epoch': 0.95}
{'loss': 0.4102, 'learning_rate': 3.3394327538883807e-06, 'epoch': 0.95}
{'loss': 0.4395, 'learning_rate': 3.3165599268069537e-06, 'epoch': 0.95}
{'loss': 0.4626, 'learning_rate': 3.293687099725526e-06, 'epoch': 0.95}
{'loss': 0.3654, 'learning_rate': 3.270814272644099e-06, 'epoch': 0.95}
{'loss': 0.3065, 'learning_rate': 3.247941445562672e-06, 'epoch': 0.95}
{'loss': 0.4303, 'learning_rate': 3.2250686184812444e-06, 'epoch': 0.95}
{'loss': 0.4421, 'learning_rate': 3.2021957913998174e-06, 'epoch': 0.95}
{'loss': 0.527, 'learning_rate': 3.1793229643183896e-06, 'epoch': 0.95}
{'loss': 0.2904, 'learning_rate': 3.1564501372369626e-06, 'epoch': 0.95}
{'loss': 0.6179, 'learning_rate': 3.133577310155535e-06, 'epoch': 0.95}
{'loss': 0.3847, 'learning_rate': 3.110704483074108e-06, 'epoch': 0.95}
{'loss': 0.481, 'learning_rate': 3.0878316559926807e-06, 'epoch': 0.95}
{'loss': 0.3336, 'learning_rate': 3.0649588289112537e-06, '

{'loss': 0.3813, 'learning_rate': 1.0064043915827997e-06, 'epoch': 0.98}
{'loss': 0.376, 'learning_rate': 9.835315645013725e-07, 'epoch': 0.98}
{'loss': 0.4958, 'learning_rate': 9.60658737419945e-07, 'epoch': 0.98}
{'loss': 0.5805, 'learning_rate': 9.377859103385179e-07, 'epoch': 0.98}
{'loss': 0.609, 'learning_rate': 9.149130832570906e-07, 'epoch': 0.99}
{'loss': 0.4996, 'learning_rate': 8.920402561756634e-07, 'epoch': 0.99}
{'loss': 0.2944, 'learning_rate': 8.691674290942361e-07, 'epoch': 0.99}
{'loss': 0.5647, 'learning_rate': 8.462946020128088e-07, 'epoch': 0.99}
{'loss': 0.4787, 'learning_rate': 8.234217749313815e-07, 'epoch': 0.99}
{'loss': 0.4541, 'learning_rate': 8.005489478499544e-07, 'epoch': 0.99}
{'loss': 0.4727, 'learning_rate': 7.77676120768527e-07, 'epoch': 0.99}
{'loss': 0.5623, 'learning_rate': 7.548032936870997e-07, 'epoch': 0.99}
{'loss': 0.4118, 'learning_rate': 7.319304666056725e-07, 'epoch': 0.99}
{'loss': 0.2601, 'learning_rate': 7.090576395242452e-07, 'epoch': 0


2021-05-27 13:06:51 Completed - Training job completed
Training seconds: 2405
Billable seconds: 2405


In [21]:
from sagemaker import TrainingJobAnalytics

# Captured metrics can be accessed as a Pandas dataframe
df = TrainingJobAnalytics(training_job_name=estimator.latest_training_job.name).dataframe()
df.head(10)

,timestamp,metric_name,value
0,0.0,loss,0.711225
1,60.0,loss,0.649542
2,120.0,loss,0.555358
3,180.0,loss,0.565467
4,240.0,loss,0.525117
5,300.0,loss,0.552800
6,360.0,loss,0.545233
7,420.0,loss,0.549250
8,480.0,loss,0.538300
9,540.0,loss,0.514083


In [22]:
df

,timestamp,metric_name,value
0,0.0,loss,0.711225
1,60.0,loss,0.649542
2,120.0,loss,0.555358
3,180.0,loss,0.565467
4,240.0,loss,0.525117
...,...,...,...
79,1260.0,epoch,0.858333
80,1320.0,epoch,0.900833
81,1380.0,epoch,0.941667
82,1440.0,epoch,0.981667
